# U.S. Transborder Freight Analysis

## Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### File Extraction

In [2]:
freight_data = pd.read_csv('unzipped_project_data/all_concat.csv',engine='python', on_bad_lines='skip')
freight_data.head(5)

MemoryError: 

In [ ]:
#Dropping source_file column
freight_data = freight_data.drop(columns=['source_file'])

# Combining Year and Month into a datetime column
freight_data['DATE'] = pd.to_datetime(freight_data['YEAR'].astype(str) + '-' + freight_data['MONTH'].astype(str))

# Sorting by date
freight_data = freight_data.sort_values('DATE')
freight_data.head(5)

## Business Questions

#### 1.	What are the overall trends in freight volume and value or revenue over time? 
#### 2.	Which transport mode (truck, rail, pipeline, air, vessel) dominated in terms of weight and value? 
#### 3.	How did trade values evolve per country (U.S.–Canada vs. U.S.–Mexico)? 
#### 4.	Which ports of entry consistently handled the highest number of shipments? 
#### 5.	What were the top commodity types traded with each country? 
#### 6.	What was the total number of shipments and value made across the years? 
#### 7.	Which freight modes had higher freight charges on average? 

### 1. Overall trends in freight volume and value over time

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
df = pd.read_csv("transborder_freight.csv", parse_dates=['Date'])
df['Year'] = df['Date'].dt.year

# Value/Weight Over Time
trend = df.groupby('Year')[['Value', 'Weight']].sum().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=trend, x='Year', y='Value', label='Total Value', color='steelblue')
sns.lineplot(data=trend, x='Year', y='Weight', label='Total Weight', color='skyblue')

# Data labels
for x, y in zip(trend['Year'], trend['Value']):
    plt.text(x, y, f'{y/1e9:.1f}B', ha='center', va='bottom', fontsize=9, color='steelblue')

for x, y in zip(trend['Year'], trend['Weight']):
    plt.text(x, y, f'{y/1e6:.1f}M', ha='center', va='top', fontsize=9, color='skyblue')

plt.title("Freight Volume & Value Over Time")
plt.ylabel("Total Value / Weight")
plt.xlabel("Year")
plt.tight_layout()
plt.show()

### 2. Transport mode dominance by weight and value

In [ ]:
# Map modes for readability
df['mode_mapped'] = df['Mode'].map({
    'Truck': 'Truck', 'Rail': 'Rail', 'Air': 'Air', 'Pipeline': 'Pipeline', 'Vessel': 'Vessel'
})

mode_summary = df.groupby('mode_mapped')[['Value', 'Weight']].sum().reset_index()

# Plot
plt.figure(figsize=(10, 6))
barplot = sns.barplot(data=mode_summary, x='mode_mapped', y='Value', palette='Blues_d')

# Data labels
for container in barplot.containers:
    barplot.bar_label(container, labels=[f'{v.get_height()/1e9:.1f}B' for v in container])

plt.title("Freight Value by Transport Mode")
plt.ylabel("Value (in USD)")
plt.xlabel("Transport Mode")
plt.tight_layout()
plt.show()


### 3. Trade Value Trend by Country

In [ ]:
df['country_mapped'] = df['Country'].map({
    'Canada': 'Canada',
    'Mexico': 'Mexico'
})

country_trend = df.groupby(['Year', 'country_mapped'])['Value'].sum().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=country_trend, x='Year', y='Value', hue='country_mapped', palette='PuBu_d')

plt.title("Trade Value Over Time: Canada vs Mexico")
plt.ylabel("Trade Value (USD)")
plt.xlabel("Year")
plt.legend(title='Country')
plt.tight_layout()
plt.show()

### 4. Top ports by shipment count

In [ ]:
top_ports = df.groupby('Port')['Shipments'].sum().nlargest(10).reset_index()

plt.figure(figsize=(12, 6))
barplot = sns.barplot(data=top_ports, x='Port', y='Shipments', palette='Blues_d')

# Add labels
barplot.bar_label(barplot.containers[0], padding=3)

plt.title("Top 10 Ports by Shipments")
plt.ylabel("Total Shipments")
plt.xlabel("Port")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 5. Top commodities per country

In [ ]:
df['commodity_mapped'] = df['Commodity'].str.title()  # Optional cleanup
top_commodities = df.groupby(['country_mapped', 'commodity_mapped'])['Value'].sum().reset_index()
top_per_country = top_commodities.sort_values(['country_mapped', 'Value'], ascending=[True, False])\
                                  .groupby('country_mapped').head(5)

# Results
print(top_per_country)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

# Load data
df = pd.read_csv("transborder_freight.csv", parse_dates=['Date'])

# Basic country mapping
df['country_mapped'] = df['Country'].map({
    'Canada': 'Canada',
    'Mexico': 'Mexico'
})

# Get top 5 commodities by value for each country
top_commodities = df.groupby(['country_mapped', 'commodity_mapped'])['Value'].sum().reset_index()
top_commodities = top_commodities.sort_values(['country_mapped', 'Value'], ascending=[True, False])
top_5_per_country = top_commodities.groupby('country_mapped').head(5)

# Pivot for stacking
pivot = top_5_per_country.pivot(index='country_mapped', columns='commodity_mapped', values='Value').fillna(0)

# Plot
pivot.plot(kind='bar', stacked=True, colormap='Blues', figsize=(10, 6))

# Data labels 
for i, row in enumerate(pivot.values):
    bottom = 0
    for j, val in enumerate(row):
        if val > 0:
            plt.text(i, bottom + val / 2, f'{val/1e9:.1f}B', ha='center', va='center', fontsize=8)
            bottom += val


plt.title("Top 5 Commodities Traded per Country (by Value)")
plt.ylabel("Trade Value (USD)")
plt.xlabel("Country")
plt.xticks(rotation=0)
plt.legend(title="Commodity", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

### 6. Total number of shipments and value across all years

In [ ]:
totals = df[['Shipments', 'Value']].sum()
print(f"Total Shipments: {totals['Shipments']:,}")
print(f"Total Value: ${totals['Value']/1e9:.2f} Billion")

### 7. Which freight modes had higher average freight charges?

In [ ]:
charge_avg = df.groupby('mode_mapped')['Freight Charges'].mean().sort_values(ascending=False)

# Pie chart
plt.figure(figsize=(8, 8))
colors = sns.color_palette("Blues", len(charge_avg))
plt.pie(charge_avg, labels=charge_avg.index,
        autopct=lambda p: f'{p:.1f}%', startangle=140,
        colors=colors, textprops={'fontsize': 11})

plt.title("Average Freight Charges by Mode")
plt.tight_layout()
plt.show()